## Dockerize Scrap Notebook

In [1]:
import docker
import os

In [2]:
client = docker.from_env()
cwd = os.getcwd()
dock_pth = os.path.join(cwd,'Dockerfile')

## Build Images

In [3]:
todo, logs = client.images.build(path=cwd, tag='todo')

In [4]:
migrations, logs = client.images.build(path=cwd, tag='migrations')


## Build Net

In [9]:
net_name = 'djang-net'
net = client.networks.create(name=net_name, driver='bridge')

## Run Containers

In [6]:
todo_ports = {'8000/tcp' : 8000}
env = ["FLASK_APP=todolist.py" , "SECRET_KEY='secret'" ]
vol = { cwd : {'bind': '/code', 'mode': 'rw'} }

In [37]:
migr_cont = client.containers.run(todo, command='flask db upgrade', name='migration',
            detach=True, network=net_name, remove=False, environment=env, volumes=vol)

In [10]:
todo_cont = client.containers.run(todo, command='gunicorn todolist:app -w 2 -b :8000',
            detach=True, network=net_name, ports=todo_ports, remove=True, environment=env, name='todolist')

In [ ]:
services:
  todolist:
    container_name: todolist
    image: todolist
    # restart: always
    env_file: .env
    command: gunicorn todolist:app -w 2 -b :8000
    ports:
      - "8000:8000"
    depends_on:
      - migration

  migration:
    container_name: migration
    build: .
    image: todolist
    env_file: .env
    command: flask db upgrade
    volumes:
      - .:/code